# Interaction with the World Homework (#3)
Python Computing for Data Science (c) J Bloom, UC Berkeley 2018

Due Tuesday 2pm, Feb 20, 2018

# 1) Monty: The Python Siri

Let's make a Siri-like program (call it Monty!) with the following properties:
   - record your voice command
   - use a webservice to parse that sound file into text
   - based on what the text, take three different types of actions:
       - send an email to yourself
       - do some math
       - tell a joke

So for example, if you say "Monty: email me with subject hello and body goodbye", it will email you with the appropriate subject and body. If you say "Monty: tell me a joke" then it will go to the web and find a joke and print it for you. If you say, "Monty: calculate two times three" it should response with printing the number 6.

Hint: you can use speed-to-text apps like Houndify (or, e.g., Google Speech https://cloud.google.com/speech/) to return the text (but not do the actions). You'll need to sign up for a free API and then follow documentation instructions for using the service within Python. 

In [1]:
# Populate our namespace
import pyaudio
import numpy as np
import wave
#from houndify import * 
import monty_key

import houndify #class provided by houndify in order to interface with thier speach to text engine
import time



In [17]:
#We define a function to record audio, we are recycling code from lecture

def record_audio(length_of_recording ,file_name='' ,path = '',sample_rate = 16000):
    """
    Takes input: 
    
    length_of_recording: the desired lenth in seconds of the recording
    path: the path for the .wav file to be saved, if not specified, it will 
    be saved in the current directory
    file_name: the name of the .wav file to be saved, if not specified, it will
    be named "new_wave.wav"
    
    sample_rate: Must be either 8kHz or 16kHz. If not specified, defaults to 16kHz
    
    Records audio from the building mic with sample rate of 44.1kHz for user defined length of time
    A .wav file is then 
    """
    
    #Check if user gave path/file name, if not, then given defaults
    if len(file_name) != 0:    
        WAVE_OUTPUT_FILENAME = path + file_name + '.wav'        
    else:
        WAVE_OUTPUT_FILENAME = path + "new_wave.wav"
        
    chunk = 1024
    #FORMAT = pyaudio.paInt32
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = sample_rate
    RECORD_SECONDS = length_of_recording                
    p = pyaudio.PyAudio()
    stream = p.open(format = FORMAT,
        channels = CHANNELS,
        rate = RATE,
        input = True,
        frames_per_buffer = chunk)
    all = []
    for i in range(0, int(RATE / chunk * RECORD_SECONDS)):
        data = stream.read(chunk)
        all.append(data)
    print("* done recording")
    stream.close()
    p.terminate()
    
    data = b"".join(all)
    wf = wave.open(WAVE_OUTPUT_FILENAME, "wb")
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(data)
    wf.close()
    
    return data
    


In [18]:
data = record_audio(6, 'joke' )

* done recording


In [19]:
#fname = 'houndify_python3_sdk_1.0.0/test_audio/whatistheweatherthere.wav'
fname = 'joke.wav'
CLIENT_ID = monty_key.Client_ID
CLIENT_KEY = monty_key.Client_Key
AUDIO_FILE = fname
BUFFER_SIZE = 512

#
# Simplest HoundListener; just print out what we receive.
# You can use these callbacks to interact with your UI.
#
class MyListener(houndify.HoundListener):
  def onPartialTranscript(self, transcript):
    #print("Partial transcript: " + transcript)
    yield
  def onFinalResponse(self, response):
    #print("Final response: " + str(response))
    yield
  def onError(self, err):
    #print("Error: " + str(err))
    yield


client = houndify.StreamingHoundClient(CLIENT_ID, CLIENT_KEY, "monty")
client.setLocation(37.388309, -121.973968)

#checks to make sure the audio meets the correct criteria to use Houndify
audio = wave.open(AUDIO_FILE)
if audio.getsampwidth() != 2:
  print("%s: wrong sample width (must be 16-bit)" % fname)
if audio.getframerate() != 8000 and audio.getframerate() != 16000:
  print("%s: unsupported sampling frequency (must be either 8 or 16 khz)" % fname)
if audio.getnchannels() != 1:
  print("%s: must be single channel (mono)" % fname)

client.setSampleRate(audio.getframerate())
client.start(MyListener())

while True:
  samples = audio.readframes(BUFFER_SIZE)
  if len(samples) == 0: break
  if client.fill(samples): break
  time.sleep(0.032) # simulate real-time so we can see the partial transcripts
     
result = client.finish() # returns either final response or error


In [20]:
result['AllResults'][0]['SpokenResponse']

'scientists use root'

# 2) Write a program that identifies musical notes from sound (AIFF) files. 

  - Run it on the supplied sound files (12) and report your program’s results. 
  - Use the labeled sounds (4) to make sure it works correctly. The provided sound files contain 1-3 simultaneous notes from different organs.
  - Save copies of any example plots to illustrate how your program works.
  
  https://piazza.com/berkeley/spring2018/ay250class13410/resources -> Homeworks -> hw3_sound_files.zip

Hints: You’ll want to decompose the sound into a frequency power spectrum. Use a Fast Fourier Transform. Be care about “unpacking” the string hexcode into python data structures. The sound files use 32 bit data. Play around with what happens when you convert the string data to other integer sizes, or signed vs unsigned integers. Also, beware of harmonics.